<a href="https://colab.research.google.com/github/Pranav1497/MLWEEK/blob/main/ml_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final.csv to final.csv


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow==2.15
import tensorflow as tf


from tensorflow import feature_column

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output







In [ ]:
training_data=pd.read_csv("final.csv")
print(training_data.columns)
training_data, test_data = train_test_split(training_data, test_size=0.2, random_state=0)

y_training_data=training_data.pop('biomass')
y_test_data=test_data.pop('biomass')


Index(['Crop', 'Light_Intensity ', 'Light_Duration', 'red_ratio', 'water',
       'nutrient', 'nitrogen', 'phosphorous', 'potassium', 'temp', 'Humidity',
       'leaf_size', 'stem_height', 'biomass'],
      dtype='object')


In [ ]:
training_data.describe()

,Light_Intensity,Light_Duration,water,nitrogen,phosphorous,potassium
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,260.937500,13.937500,5.828125,168.750000,41.562500,158.437500
std,72.438911,0.873212,18.455288,35.939764,10.911576,51.306879
min,150.000000,13.000000,0.750000,100.000000,20.000000,70.000000
25%,243.750000,13.000000,0.750000,150.000000,30.000000,137.500000
50%,250.000000,14.500000,1.250000,175.000000,50.000000,150.000000
75%,300.000000,14.500000,1.500000,200.000000,50.000000,200.000000
max,400.000000,15.000000,75.000000,200.000000,50.000000,250.000000


In [ ]:
catogerical_columns=["Crop","red ratio","nutrient","Humidity","leaf size","stem height","biomass"]
numeric_columns=["nitrogen", "phosphorous","potassium",'Light Intensity',"Light Duration","red ratio","water",
                     "temp"]

feature_columns = []
for feature_name in catogerical_columns :
  if feature_name in training_data.columns:
    training_data[feature_name] = training_data[feature_name].astype(str)
    vocabulary = training_data[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name.replace(" ","_"), vocabulary))

for feature_name in numeric_columns:
  if feature_name in training_data.columns:
    if feature_name=="Light Duration":
      feature_columns.append(tf.feature_column.categorical_column_with_hash_bucket(
          feature_name.replace(" ", "_"), hash_bucket_size=1000))
    else:
      training_data[feature_name] = training_data[feature_name].astype(float)



print(feature_columns)



[VocabularyListCategoricalColumn(key='Crop', vocabulary_list=('2', '13', '10', '5', '1', '4', '15', '11', '8', '3', '0', '7', '12', '14', '6', '9'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='nutrient', vocabulary_list=('7', '8', '5', '0', '3', '1', '4', '9', '10', '6', '2'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='Humidity', vocabulary_list=('1', '3', '2', '0', '4'), dtype=tf.string, default_value=-1, num_oov_buckets=0)]


In [ ]:
def make_input_fn(data_df, label_df, num_epochs=50, shuffle=True, batch_size=21):
  def input_function():
    nonlocal label_df
    nonlocal data_df
    data_df_copy=data_df.copy()

    if label_df.dtype != 'float32':
      label_df = label_df.astype(str).str.replace('[^0-9.-]', 'NaN', regex=True)


      label_df = pd.to_numeric(label_df, errors='coerce').astype('float32')
      data_df = data_df[~label_df.isnull()]
      label_df = label_df.dropna()

    else:
      label_df_copy=label_df

    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(training_data, y_training_data)
eval_input_fn = make_input_fn(test_data, y_test_data, num_epochs=1, shuffle=False)


In [ ]:
linear_regression = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [ ]:
linear_regression.train(train_input_fn)
result = linear_regression.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])